In [20]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100



In [54]:
# Load Datasets

df_1 = pd.read_csv('player_profile_df_1.csv')
df_2 = pd.read_csv('player_profile_df_2.csv')
df_3 = pd.read_csv('player_profile_df_3.csv')

player_injuries_df = pd.read_csv('player_injuries_df.csv')
player_weights_df = pd.read_csv('player_weights.csv')

In [55]:
profile_df = pd.concat([df_1, df_2])
profile_df = pd.concat([profile_df, df_3])

In [56]:
# pd. set_option('display.max_rows', 5000)

# profile_df['Date of birth:'] = profile_df['Date of birth:'].fillna('') 
# profile_df[profile_df['Date of birth:'].str.contains('Executive assist...')]


# df = pd.DataFrame(profile_df['club:'].value_counts()).reset_index()['index']
# list(df)
# pd.DataFrame(df).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/jobs.csv', index=False)

In [57]:
# profile_df[profile_df['Place of birth:'].str.contains('Head of first')]
# profile_df.loc[profile_df['Place of birth:'].fillna('').str.contains("Head of first", case=False)]

#  'Head of first te...'
#  'Director of Foot...'
#  'Member of the Su...'
#  'Chief Executive ...'


In [58]:
# Merge Profile Information with Injuries

profile_injuries_df = pd.merge(left=player_injuries_df, right=profile_df, left_on=['TMId', 'name'], right_on=['TMId', 'name'], how='left')

In [26]:
profile_injuries_df['Foot:'].value_counts(dropna=False)

right            37425
left             13233
both              3233
NaN               1322
Italy                2
Benin  France        1
Name: Foot:, dtype: int64

In [63]:
from unidecode import unidecode
import string

import re
  
def remove(string):
    pattern = re.compile(r'\s+')
    return re.sub(pattern, '', string)

profile_injuries_df['UID'] = profile_injuries_df['name'].fillna('').str.lower().str.replace(' ','').str.strip() + \
                             profile_injuries_df['Citizenship:'].fillna('').str.lower().str.replace(' ','').str.strip() + \
                             profile_injuries_df['Foot:'].fillna('').str.lower().str.replace(' ','').str.strip()
profile_injuries_df['UID'] = profile_injuries_df['UID'].astype(str).apply(unidecode).apply(remove)
# profile_injuries_df['UID'] = profile_injuries_df['UID'].str.replace('[^\w\s]','')

profile_injuries_df['UID'] = profile_injuries_df['UID'].str.replace('[{}]'.format(string.punctuation), '')

profile_injuries_df.head()

<ipython-input-63-83537511564c>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  profile_injuries_df['UID'] = profile_injuries_df['UID'].str.replace('[{}]'.format(string.punctuation), '')


,Season,Injury,from,until,Days,Games missed,TMId,name,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,UID
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,petrcechczechrepublicleft
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,petrcechczechrepublicleft
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,petrcechczechrepublicleft
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,petrcechczechrepublicleft
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,petrcechczechrepublicleft


In [64]:
profile_injuries_df.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/profile_injuries_df.csv')

In [28]:
player_weights_df = player_weights_df[player_weights_df['year'] != 'FIFA 07']

player_weights_df['UID'] = player_weights_df['Full Name'].fillna('').str.lower().str.replace(' ','') \
                         + player_weights_df['Nationality'].fillna('').str.lower().str.replace(' ','') \
                         + player_weights_df['Strong foot'].fillna('').str.lower().str.replace(' ','')
player_weights_df['UID'] = player_weights_df['UID'].astype(str).apply(unidecode).apply(remove)
# player_weights_df['UID'] = player_weights_df['UID'].str.replace('[^\w\s]','')

player_weights_df['UID'] = player_weights_df['UID'].str.replace('[{}]'.format(string.punctuation), '')

player_weights_df.head()


<ipython-input-28-a599083cd881>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  player_weights_df['UID'] = player_weights_df['UID'].str.replace('[{}]'.format(string.punctuation), '')


,Name,Age,Overall Rating,Potential,Team,Contract expiry,Height,Weight,Strong foot,Value,Full Name,Nationality,date,year,UID
0,L. Messi,34.0,93.0,93.0,Paris Saint-Germain,2021 ~ 2023,170cm,72kg,Left,€78M,Lionel Messi,Argentina,"Nov 23, 2021",FIFA 22,lionelmessiargentinaleft
1,R. Lewandowski,32.0,92.0,92.0,FC Bayern München,2014 ~ 2023,185cm,81kg,Right,€119.5M,Robert Lewandowski,Poland,"Nov 23, 2021",FIFA 22,robertlewandowskipolandright
2,K. De Bruyne,30.0,91.0,91.0,Manchester City,2015 ~ 2025,181cm,70kg,Right,€125.5M,Kevin De Bruyne,Belgium,"Nov 23, 2021",FIFA 22,kevindebruynebelgiumright
3,Cristiano Ronaldo,36.0,91.0,91.0,Manchester United,2021 ~ 2023,187cm,83kg,Right,€45M,C. Ronaldo dos Santos Aveiro,Portugal,"Nov 23, 2021",FIFA 22,cronaldodossantosaveiroportugalright
4,Neymar Jr,29.0,91.0,91.0,Paris Saint-Germain,2017 ~ 2025,175cm,68kg,Right,€129M,Neymar da Silva Santos Jr.,Brazil,"Nov 23, 2021",FIFA 22,neymardasilvasantosjrbrazilright


In [ ]:
player_weights_df.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/player_weights_df.csv')

In [29]:
uid_player_profiles = list(set(profile_injuries_df['UID']))
uid_player_weights = list(set(player_weights_df['UID']))

# uid_player_profiles

In [66]:
# print(uid_player_profiles)

In [30]:
len(uid_player_profiles), len(uid_player_weights)

(9170, 10945)

In [31]:
from difflib import SequenceMatcher
from functools import partial

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [50]:
# Creating first part of Similarity data frame (Sravan)

similarity_df = pd.DataFrame([])

for profile in uid_player_profiles[0:100]:
    for weight in uid_player_weights[:100]:
        similarity_df.loc[profile, weight] = similar(profile, weight)



In [51]:
similarity_df

,mergimmavrajgermanyleft,samikhediragermanyleft,adailtonbrazilleft,juniorstanislasenglandright,jeanlucdompediarrafranceright,brunoribeiroingraciabrazilright,piotrzielinskipolandright,pedroriosspainright,mbayeniangfranceright,aitorlopezrekartespainright,miguelflanospainright,danielpavlovicswitzerlandleft,rachidghezzalfranceleft,fabianoellerbrazilleft,alejandropozopozospainright,dennissrbenygermanyright,emremorturkeyleft,maximilianbeistergermanyleft,leonardobalerdiargentinaright,rubenmartinezandradespainright,luisfernandezspainleft,ivaringimarssonicelandright,alexanderbadegermanyright,jonathanligalifranceright,diegohernandezbarriusospainright,georgethomaswalesright,daniellarssonswedenright,emmanuelmayukazambiaright,keplerleveranlimaferreiraportugalright,francomlinarcroatiaright,massadiohaidarasenegalleft,juanpablopinocolombiaright,jairoizquierdogonzalezspainleft,stevesidwellenglandright,raulrodrigueznavarrospainright,lukeshawenglandleft,benjaminkohlergermanyleft,romeocastelennetherlandsright,nemanjaradojaserbiaright,hectorspainleft,robsondesouzabrazilright,armandogoncalvesteixeiraportugalright,jordigomezspainleft,emmanuelemenikenigeriaright,alexanderbugeragermanyleft,juanjosecamachobarnolaspainleft,thibaultgiressefranceleft,adriansempercroatiaright,alvarovadillocifuentesspainright,dominickdrexlergermanyright,francojaraargentinaright,javifuegospainright,urosradakovicserbialeft,matshummelsgermanyright,lennartgrillgermanyright,joshuameesgermanyright,joseignaciomartinezgarciaspainleft,vasilispliatsikasgreeceright,marcokurthgermanyleft,liamhendersonscotlandright,giovannimarcheseitalyleft,jackcollisonenglandright,kajsierhuisnetherlandsright,connorgoldsonenglandright,seydoukeitamalileft,ricardofatysenegalright,gregoirepuelfranceright,kerimfreiturkeyright,gaetandeneuvefranceright,alexandreraineaufranceleft,danielolmocarvajalspainright,facundomedinaargentinaleft,alessandronestaitalyright,harisseferovicswitzerlandright,philipplienhartaustriaright,sergiobuenacasaalbaspainright,lennartthygermanyright,alexandermerkelgermanyright,sambouyatabaremaliright,alexandermanningeraustriaright,lukevarneyenglandright,kadermanganesenegalright,damjandjokoviccroatialeft,robholdingenglandright,youneskaboulfranceright,massihwasseycanadaleft,giuseppecolucciitalyright,ianashbeeenglandright,jacobbutterfieldenglandright,miguelangelodasilvarochaportugalright,gastonsilvauruguayleft,abdoulrazzaguicamaraguinealeft,carlmedjanialgeriaright,jensgrahlgermanyright,beteedelapoulaedimaarmeniaright,abdoulayebasenegalright,jonaslossldenmarkright,tayetaiwonigerialeft,mathieudeplagnefranceright,saulniguezesclapezspainleft
giacomoolzeritalyleft,0.500000,0.418605,0.358974,0.208333,0.280000,0.269231,0.304348,0.200000,0.333333,0.250000,0.238095,0.440000,0.409091,0.558140,0.166667,0.222222,0.421053,0.408163,0.240000,0.235294,0.372093,0.333333,0.260870,0.347826,0.226415,0.372093,0.311111,0.217391,0.271186,0.355556,0.297872,0.212766,0.423077,0.222222,0.196078,0.300000,0.478261,0.320000,0.222222,0.277778,0.222222,0.275862,0.450000,0.208333,0.425532,0.384615,0.391304,0.311111,0.188679,0.291667,0.311111,0.250000,0.272727,0.227273,0.311111,0.232558,0.290909,0.285714,0.523810,0.340426,0.608696,0.355556,0.291667,0.391304,0.500000,0.272727,0.363636,0.292683,0.311111,0.425532,0.326531,0.510638,0.391304,0.274510,0.208333,0.320000,0.232558,0.250000,0.181818,0.235294,0.232558,0.266667,0.260870,0.325581,0.272727,0.372093,0.478261,0.285714,0.285714,0.310345,0.465116,0.392157,0.318182,0.333333,0.192308,0.272727,0.279070,0.439024,0.297872,0.333333
thomasblomeyergermanyboth,0.458333,0.425532,0.279070,0.269231,0.259259,0.035714,0.240000,0.045455,0.347826,0.346154,0.260870,0.296296,0.291667,0.255319,0.269231,0.489796,0.238095,0.490566,0.370370,0.218182,0.255319,0.269231,0.520000,0.320000,0.280702,0.468085,0.163265,0.280000,0.222222,0.285714,0.235294,0.313725,0.285714,0.326531,0.036364,0.272727,0.480000,0.296296,0.244898,0.300000,0.040816,0.258065,0.272727,0.423077,0.509804,0.250000,0.360000,0.204082,0

In [52]:
# saving similarity_df to CSV

# similarity_df.to_csv('Similarity_df_1_test.csv')

In [49]:
text = pd.read_csv('Similarity_df_1_test.csv')
text

,Unnamed: 0,mergimmavrajgermanyleft,samikhediragermanyleft,adailtonbrazilleft,juniorstanislasenglandright,jeanlucdompediarrafranceright,brunoribeiroingraciabrazilright,piotrzielinskipolandright,pedroriosspainright,mbayeniangfranceright,aitorlopezrekartespainright
0,giacomoolzeritalyleft,0.500000,0.418605,0.358974,0.208333,0.280000,0.269231,0.304348,0.200000,0.333333,0.250000
1,thomasblomeyergermanyboth,0.458333,0.425532,0.279070,0.269231,0.259259,0.035714,0.240000,0.045455,0.347826,0.346154
2,carlosakapoequatorialguineaspainright,0.166667,0.237288,0.327273,0.437500,0.333333,0.441176,0.419355,0.500000,0.413793,0.531250
3,pedroriosspainright,0.285714,0.341463,0.378378,0.521739,0.500000,0.480000,0.454545,1.000000,0.450000,0.608696
4,rogerespinozahondurasunitedstatesleft,0.300000,0.305085,0.363636,0.312500,0.181818,0.235294,0.322581,0.321429,0.206897,0.343750
5,kyletaylorenglandswitzerlandright,0.142857,0.181818,0.117647,0.500000,0.419355,0.343750,0.379310,0.346154,0.518519,0.366667
6,marcothiedegermanyright,0.478261,0.533333,0.146341,0.360000,0.423077,0.296296,0.458333,0.476190,0.454545,0.400000
7,miguelflanospainright,0.318182,0.325581,0.358974,0.416667,0.480000,0.384615,0.478261,0.600000,0.523810,0.541667
8,markeletxeberriaspainright,0.285714,0.208333,0.227273,0.415094,0.327273,0.491228,0.392157,0.577778,0.510638,0.490566
9,dennissrbenygermanyright,0.468085,0.478261,0.142857,0.588235,0.301887,0.472727,0.408163,0.465116,0.444444,0.274510


In [67]:
# Renzo

similarity_df = pd.DataFrame([])

for profile in uid_player_profiles[3000:6000]:
    for weight in uid_player_weights:
        similarity_df.loc[profile, weight] = similar(profile, weight)



KeyboardInterrupt: 

In [ ]:
similarity_df.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/Similarity_df_2.csv')

In [ ]:
# Victor

similarity_df = pd.DataFrame([])

for profile in uid_player_profiles[6000:-1]:
    for weight in uid_player_weights:
        similarity_df.loc[profile, weight] = similar(profile, weight)
        
similarity_df.to_csv('Similarity_df_3.csv', index=False)